In [2]:
import os 
import pandas as pd
import numpy as np

In [3]:
os.getcwd()

'/Users/aravindreddypasham/Downloads'

In [4]:
os.chdir('/Users/aravindreddypasham/Desktop/Python/instacart')

In [5]:
ls

Insta cart anlysis  (1).ipynb  order_products__prior.csv
aisles.csv                     order_products__train.csv
aisles.csv.zip                 orders.csv
departments.csv                products.csv


In [6]:
order_products_prior = pd.read_csv('order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})
order_products_train = pd.read_csv('order_products__train.csv', dtype = {
    'order_id': np.int32,
    'product_id':np.uint16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8,
    })
order = pd.read_csv('orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})
products =  pd.read_csv( 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8,
        'product_name' : 'category'})
aisles = pd.read_csv("aisles.csv", dtype= { 
    'aisle_id':np.int8,
    'aisle' :'category'})
departments = pd.read_csv("departments.csv", dtype = {
    'department_id':np.int32,
    'department':'category'})
    

In [8]:
import pickle

In [10]:
goods = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')


In [13]:
goods.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,2,All-Seasons Salt,104,13,pantry,spices seasonings
2,3,Robust Golden Unsweetened Oolong Tea,94,7,beverages,tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen,frozen meals
4,5,Green Chile Anytime Sauce,5,13,pantry,marinades meat preparation


In [12]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [15]:
import gc

In [20]:
log = pd.concat([order_products_train,order_products_prior
                ] , ignore_index = 1)

In [22]:
log.sort_values(['order_id','add_to_cart_order'],inplace = True)


In [23]:
log.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [32]:
log.reset_index(drop=1, inplace=True)

In [34]:
log = pd.merge(log, goods, on = 'product_id', how = 'left')

In [37]:
log = pd.merge(log, order, on = 'order_id', how = 'left')

In [40]:
log['order_number_rev']=log.groupby('user_id').order_number.transform(np.max)-log.order_number

In [42]:
log.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_number_rev
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt,112108,train,4,4,10,9.0,0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,112108,train,4,4,10,9.0,0
2,1,10246,3,0,Organic Celery Hearts,83,4,produce,fresh vegetables,112108,train,4,4,10,9.0,0
3,1,49683,4,0,Cucumber Kirby,83,4,produce,fresh vegetables,112108,train,4,4,10,9.0,0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood,112108,train,4,4,10,9.0,0


In [45]:
order_product = log.groupby('order_id').product_name.apply(list).reset_index()

In [46]:
order_product.head()

,order_id,product_name
0,1,"[Bulgarian Yogurt, Organic 4% Milk Fat Whole M..."
1,2,"[Organic Egg Whites, Michigan Organic Kale, Ga..."
2,3,[Total 2% with Strawberry Lowfat Greek Straine...
3,4,"[Plain Pre-Sliced Bagels, Honey/Lemon Cough Dr..."
4,5,"[Bag of Organic Bananas, Just Crisp, Parmesan,..."


In [48]:
order_tbl = pd.merge(order, order_product, on = 'order_id', how = 'left')

In [51]:
order_tbl.sort_values(['user_id','order_number'], inplace = True)

In [53]:
order_tbl.reset_index(drop = 1, inplace = True)

In [54]:
order_tbl.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
product_name               75000
dtype: int64

In [55]:
log.isnull().sum()


order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
product_name                    0
aisle_id                        0
department_id                   0
department                      0
aisle                      885247
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
order_number_rev                0
dtype: int64

In [56]:
order_tbl.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name
0,2539329,1,prior,1,2,8,NaN,"[Soda, Organic Unsweetened Vanilla Almond Milk..."
1,2398795,1,prior,2,3,7,15.0,"[Soda, Pistachios, Original Beef Jerky, Bag of..."
2,473747,1,prior,3,3,12,21.0,"[Soda, Original Beef Jerky, Pistachios, Organi..."
3,2254736,1,prior,4,4,7,29.0,"[Soda, Original Beef Jerky, Pistachios, Organi..."
4,431534,1,prior,5,4,15,28.0,"[Soda, Original Beef Jerky, Pistachios, Organi..."


In [57]:
order_tbl = pd.merge(order_tbl, log[['order_id','order_number_rev']].drop_duplicates(), on = 'order_id',how = 'left')

In [63]:
order_tbl.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
product_name               75000
order_number_rev               0
dtype: int64

In [60]:
order_tbl.order_number_rev = order_tbl.order_number_rev.fillna(-1).astype(int)

In [64]:
order_tbl['order_number_rev']=order_tbl.groupby('user_id').order_number.transform(np.max)-order_tbl.order_number

In [66]:
order_tbl['days_since_first_order']=order_tbl.groupby('user_id').days_since_prior_order.cumsum()

In [67]:
def set_diff(items1, items2):
    if isinstance(items1,float) or isinstance(items2, float):
        return items1
    return[i1 for i1 in items1 if i1 not in items1]

In [68]:
def same_products(items1, items2):
    if isinstance(items1,float) or isinstance(items2, float):
        return[]
    return[i1 for i1 in items1 if i1 in items2]

In [70]:
order_tbl['t-1_product_name']=order_tbl.groupby('user_id')['product_name'].shift(1)
order_tbl['set_diff_products']=order_tbl.apply(lambda x: set_diff(x['product_name'],x['t-1_product_name']),axis =1)
order_tbl['same_products'] = order_tbl.apply(lambda x: same_products(x['product_name'], x['t-1_product_name']),axis = 1)

In [74]:
order_tbl.to_pickle('order_tbl.p')

In [75]:
goods.to_pickle('goods.p')

In [76]:
order_aisle = pd.crosstab(log['order_id'],log['aisle_id']).add_prefix('aisle_').reset_index()

In [78]:
order_department = pd.crosstab(log['order_id'],log['department_id']).add_prefix('department_').reset_index()

In [82]:
order_aisle = pd.merge(order_aisle, order_department, on = 'order_id', how = 'left')

In [86]:
order_aisle.to_pickle('order_aisle_department.p')

OSError: [Errno 22] Invalid argument

In [87]:
log_ = log.loc[log.reordered ==1]
order_reorderd = log_.groupby('order_id').product_id.apply(list).reset_index()

In [88]:
order_reorderd.head()

,order_id,product_id
0,1,"[49302, 11109, 43633, 22035]"
1,2,"[33120, 28985, 45918, 17794, 40141, 1819]"
2,3,"[33754, 24838, 17704, 21903, 17668, 46667, 174..."
3,4,"[26434, 39758, 27761, 10054, 21351, 22598, 348..."
4,5,"[13176, 15005, 47329, 27966, 23909, 48370, 132..."


In [90]:
order_reorderd.to_pickle('order_reorderd.p')


In [91]:
from itertools import chain 


In [92]:
order_tbl = pd.read_pickle('order_tbl.p')

In [93]:
order_tbl = order_tbl.loc[order_tbl.eval_set!='test']

In [97]:
goods = pd.read_pickle('goods.p')

In [99]:
goods_di = {}
for k,v in zip(goods.product_name, goods.product_id):
    goods_di[k] = v

In [101]:
def sum_list(x):
    return list(chain.from_iterable(x))

In [102]:
def to_unique(lists):
    li = sum_list(lists)
    return list(set(li))

In [103]:
def to_ids(names):
    ids = [goods_di[n] for n in names]
    return ids

In [104]:
user_hist = order_tbl.groupby('user_id').product_name.apply(to_unique).reset_index()
user_hist['product_id'] = user_hist.product_name.map(to_ids)

In [105]:
user_hist.head()

,user_id,product_name,product_id
0,1,"[Organic Unsweetened Almond Milk, Cinnamon Toa...","[35951, 13032, 13176, 10258, 10326, 39657, 417..."
1,2,"[Smooth Operator Natural Peanut Butter, Truffl...","[36287, 48099, 4071, 5907, 18523, 2002, 44303,..."
2,3,"[All Natural No Stir Creamy Almond Butter, Blu...","[1819, 15143, 248, 12845, 22035, 8021, 42265, ..."
3,4,"[Vegetable Classics Vegetable Soup, Yellow Cor...","[1200, 26576, 25623, 7350, 11865, 2707, 35469,..."
4,5,"[Lightly Salted Baked Snap Pea Crisps, Snow Pe...","[13870, 13988, 48775, 16168, 15349, 18761, 201..."


In [106]:
user_hist.to_pickle('user_order.p')